In [ ]:
import sys

sys.path.append(".../pipeline_imaging")  #'/path/to/your/project'

In [ ]:
import pydicom
import imaging_utils
import os

In [ ]:
from imaging_flio_root import Flio

flio_instance = Flio()

# Folder structures

In [ ]:
# Pooled date folders across three sites
# step1 (pooled)/
#  ├── UW_FLIO_20240730-20240827/
#  └── UW_FLIO_20240730-20240827/
#      ├──457_AIREADI_1375
#      └──268_AIREADI_4248


# => flio_instance.organize =>

# Files orangnized by per scan
# step2/
#  ├── flio_268_AIREADI_4248_2024_07_01-13_00_29_OD
#  ├── flio_268_AIREADI_4248_2024_07_01-13_03_01_OS
#  ├── ...
#  └── flio_459_AIREADI_1376_2024_07_25-13_02_15_OS
#      ├── AF_Intensity_LONG-Channel.bmp
#      ├── AF_Intensity_Long+Short_Channels.bmp
#      ├── AF_Intensity_Short-Channel.bmp
#      ├── measurement_info.html
#      └── Measurement.sdt

#  => spectralis_instance.convert1 =>

# From a scan, dicom files are created
# step3/
#     ├── 1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     ├── 1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     └── ... (additional DICOM files)


#  => flio_instance.converted2 =>

# From a dicom file, NEMA compliant DICOM files are created
# step4/
#     ├── converted_1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     ├── converted_1375_flio_long_wavelength_l_1.2.826.0.1.3680043.8.498.73459378417040338489805556532204911380.dcm
#     └── ... (additional DICOM files)

#  => imaging_utils.format_file =>

# De-identified again, file renamed, and organized in a final structure for data relase
# step5/
#    ├── retinal_flio/
#         └── heidelberg_flio/
#               ├── 1001
#               ├── 1002
#               ├── ... (additional folders)
#               ├── 1003
#                     └── 1375_flio_long_wavelength_r_1.2.826.0.1.3680043.8.498.12481560985877485897830542137951711344.dcm
#                     └── 1375_flio_long_wavelength_r_1.2.826.0.1.3680043.8.498.12481560985877485897830542137951711344.dcm


# Organize by scan

In [ ]:
input = ".../step1/UW_FLIO_20240730-20240827"
output = ".../step2/FLIO"

organize_result = flio_instance.organize(input, output)

# Convert1 - convert a folder (worth one scan) to two dicom files

In [ ]:
input = ".../step2"
output = ".../step3"
jsonpath = ".../imaging/flio_uid_data.json"

flio_folderlist = imaging_utils.list_subfolders(input)

for folder in flio_folderlist:
    if "flio" in folder:
        cov_result = flio_instance.convert1(folder, output, jsonpath)

# Convert 2 - convert dicom files to NEMA compliant dicom files

In [ ]:
input = ".../step3"
output = ".../step4"

filtered_list = imaging_utils.get_filtered_file_names(input)

for file in filtered_list:
    if "flio" in file:
        cov_result = flio_instance.convert2(file, output)

# Files are de-identified again, file renamed, and organized in a final structure for data relase

In [ ]:
input = ".../step4"
output = ".../step5"
metadata_folder = ".../step_metadata"

filelist = imaging_utils.get_filtered_file_names(input)

for file in filelist:
    if "flio" in file:
        full_file_path = imaging_utils.format_file(file, output)
        flio_instance.metadata(full_file_path, metadata_folder)